### Grammar

S -> NP VP<br/>
S -> S PP<br/>
<br/>
NP -> Det N<br/>
NP -> Det Adj N<br/>
NP -> N<br/>
NP -> Adj N<br/>
NP -> NP PP<br/>
NP -> NP Conj NP<br/>
NP -> VP<br/>
<br/>
VP -> Aux VP<br/>
VP -> Cop Adj<br/>
VP -> V<br/>
VP -> V NP<br/>
VP -> V Adv<br/>
VP -> V NP Adv<br/>
VP -> V PP<br/>
VP -> VP PP<br/>
<br/>
PP -> P NP<br/>
<br/>
Det -> 'the' | 'The'<br/>
N -> 'planes' | 'parents' | 'bride' | 'groom'<br/>
Adj -> 'flying' | 'dangerous'<br/>
Aux -> 'can' | 'were'<br/>
Cop -> 'be'<br/>
V -> 'loves' | 'flying'<br/>
P -> 'of' | 'than'<br/>
Conj -> 'and'<br/>
Adv -> 'more'<br/>



In [24]:
from nltk import CFG, ChartParser

grammar = CFG.fromstring("""
S -> NP VP
S -> S PP

NP -> Det N
NP -> Det Adj N
NP -> N
NP -> Adj N
NP -> NP PP
NP -> NP Conj NP
NP -> VP

VP -> Aux VP
VP -> Cop Adj
VP -> V
VP -> V NP
VP -> V Adv
VP -> V NP Adv
VP -> V PP
VP -> VP PP

PP -> P NP

Det -> 'the' | 'The'
N -> 'planes' | 'parents' | 'bride' | 'groom'
Adj -> 'flying' | 'dangerous'
Aux -> 'can' | 'were'
Cop -> 'be'
V -> 'loves' | 'flying'
P -> 'of' | 'than'
Conj -> 'and'
Adv -> 'more'
""")

parser = ChartParser(grammar)

raw_sentences = [
    "Flying planes can be dangerous",
    "The parents of the bride and the groom were flying",
    "The groom loves dangerous planes more than the bride"
]

for s in raw_sentences:
    tokens = s.lower().split()
    print("\nSentence:", s)
    trees = list(parser.parse(tokens))
    if not trees:
        print("No valid parse found.")
    else:
        print(f"Found {len(trees)} parse(s):")
        for i, t in enumerate(trees, 1):
            print(f"\n--- Parse {i} ---")
            print(t)
            t.pretty_print()


Sentence: Flying planes can be dangerous
Found 2 parse(s):

--- Parse 1 ---
(S
  (NP (Adj flying) (N planes))
  (VP (Aux can) (VP (Cop be) (Adj dangerous))))
                   S                   
         __________|___                 
        |              VP              
        |           ___|___             
        NP         |       VP          
   _____|____      |    ___|______      
 Adj         N    Aux Cop        Adj   
  |          |     |   |          |     
flying     planes can  be     dangerous


--- Parse 2 ---
(S
  (NP (VP (V flying) (NP (N planes))))
  (VP (Aux can) (VP (Cop be) (Adj dangerous))))
                   S                   
         __________|___                 
        NP             |               
        |              |                
        VP             VP              
   _____|____       ___|___             
  |          NP    |       VP          
  |          |     |    ___|______      
  V          N    Aux Cop        Adj   
  |  

In [5]:
import spacy
from spacy.cli import download
from spacy import displacy

# Ensure English model is installed
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

texts = [
    "Flying planes can be dangerous.",
    "The parents of the bride and the groom were flying.",
    "The groom loves dangerous planes more than the bride."
]

for sent in texts:
    print("\n" + "═" * 80)
    print(f"Sentence: {sent}")
    print("─" * 80)

    doc = nlp(sent)

    print(f"{'TOKEN':<15} {'POS':<10} {'DEP':<12} {'HEAD':<15} CHILDREN")
    print("-" * 80)

    for token in doc:
        children = ", ".join([child.text for child in token.children]) or "-"
        print(f"{token.text:<15} {token.pos_:<10} {token.dep_:<12} {token.head.text:<15} {children}")

    print("\nDependency Tree:")
    for token in doc:
        indent = "  " * (len(list(token.ancestors)))
        print(f"{indent}↳ {token.text} ({token.dep_}) → {token.head.text}")




════════════════════════════════════════════════════════════════════════════════
🧩 Sentence: Flying planes can be dangerous.
────────────────────────────────────────────────────────────────────────────────
TOKEN           POS        DEP          HEAD            CHILDREN
--------------------------------------------------------------------------------
Flying          VERB       amod         planes          -
planes          NOUN       nsubj        be              Flying
can             AUX        aux          be              -
be              AUX        ROOT         be              planes, can, dangerous, .
dangerous       ADJ        acomp        be              -
.               PUNCT      punct        be              -

Dependency Tree:
    ↳ Flying (amod) → planes
  ↳ planes (nsubj) → be
  ↳ can (aux) → be
↳ be (ROOT) → be
  ↳ dangerous (acomp) → be
  ↳ . (punct) → be

════════════════════════════════════════════════════════════════════════════════
🧩 Sentence: The parents of the brid

### Application: Question Answering Systems

Question Answering (QA) systems—like virtual assistants or search engines that respond to natural language queries—rely heavily on syntactic and dependency parsing to correctly interpret meaning and extract relevant information. Parsing enables the system to move beyond surface-level keyword matching and instead understand *who did what to whom* within a sentence.

**Syntactic parsing** provides phrase-structure information (e.g., noun phrases, verb phrases, prepositional phrases). This helps isolate key sentence components such as subjects, predicates, and objects. For instance, in the question *"Who loves dangerous planes?"*, syntactic parsing identifies *"loves dangerous planes"* as the verb phrase and allows the system to focus on the subject that performs this action.

**Dependency parsing** goes further by defining grammatical relations between individual words. It reveals the functional structure of a sentence through labeled dependencies (like `nsubj`, `dobj`, `amod`, `prep`, etc.). In the example above, dependency parsing would show that *"groom"* is the nominal subject (`nsubj`) of *"loves"*, and *"planes"* is its direct object (`dobj`), modified by *"dangerous"* (`amod`). Such relationships are critical for understanding meaning.

In QA systems, dependency structures help link a user’s question to the correct piece of information in a database or corpus. For example, given *"Who loves dangerous planes more than the bride?"*, dependency parsing allows the system to interpret the comparative structure and identify that *"the groom"* is the subject of the verb *"loves"*. This understanding ensures the system retrieves the right answer rather than relying on ambiguous word proximity.

By combining syntactic and dependency parsing, QA systems can perform more accurate reasoning, disambiguate meanings, handle complex clauses, and deliver responses that reflect a genuine understanding of sentence structure rather than pattern matching.
